<a href="https://colab.research.google.com/github/riemannruiz/MS_Modelado_Predictivo/blob/master/P0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Índices de similitud**
En este ejercicio tiene como objetivo explorar los índices de similitud usando diferentes tipos de datos. Para este ejemplo se utilizan un conjunto de datos generados por alumnos de la universidad, a quienes les fue solicitado la calificación de un conjunto de películas en una escala de 1 a 5 estrellas.

Cargar las librerías necesarias para ejecutar el ejercicio.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm # metricas de similitud
import scipy.spatial.distance as sc # metricas de distancia

Si el ejercicio está ejecutandose en la plataforma **Google Colab**, se debe de cargar los datos a la plataforma para poder acceder a los mismos. La carga de los datos se realizá con el código siguiente.

Si el ejercicio se está ejecutando de forma local se debe omitir el código siguiente.

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Test de películas.xlsx to Test de películas.xlsx
User uploaded file "Test de películas.xlsx" with length 24470 bytes


Leer e importar los datos mediante la paquetería pandas y los almacenamos en un DataFrame.

In [7]:
file_path = 'Test de películas.xlsx'
data = pd.read_excel(file_path,encoding='latin_1',index_col=0)
data.head()

,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Total de puntos,Comentarios del cuestionario,Sobrenombre,Puntos: Sobrenombre,Comentarios: Sobrenombre,Inteligencia Artificial,Puntos: Inteligencia Artificial,Comentarios: Inteligencia Artificial,Super Man,Puntos: Super Man,Comentarios: Super Man,E.T.,Puntos: E.T.,Comentarios: E.T.,Interestelar,Puntos: Interestelar,Comentarios: Interestelar,Mr. Nobody,Puntos: Mr. Nobody,Comentarios: Mr. Nobody,Star Wars,Puntos: Star Wars,Comentarios: Star Wars,V de Vendeta,Puntos: V de Vendeta,Comentarios: V de Vendeta,Avatar,Puntos: Avatar,Comentarios: Avatar,Contacto,Puntos: Contacto,Comentarios: Contacto,El Origen (Inception),Puntos: El Origen (Inception),Comentarios: El Origen (Inception),Matrix,...,Comentarios: Monthy Python,American Pie,Puntos: American Pie,Comentarios: American Pie,Nosotros los nobles,Puntos: Nosotros los nobles,Comentarios: Nosotros los nobles,La mascara,Puntos: La mascara,Comentarios: La mascara,El señor de los anillos,Puntos: El señor de los anillos,Comentarios: El señor de los anillos,Harry Potter,Puntos: Harry Potter,Comentarios: Harry Potter,Blanca Nieves,Puntos: Blanca Nieves,Comentarios: Blanca Nieves,Big fish,Puntos: Big fish,Comentarios: Big fish,Narnia,Puntos: Narnia,Comentarios: Narnia,El laberinto del fauno,Puntos: El laberinto del fauno,Comentarios: El laberinto del fauno,The shape of water,Puntos: The shape of water,Comentarios: The shape of water,Aquaman,Puntos: Aquaman,Comentarios: Aquaman,El lobo de wall street,Puntos: El lobo de wall street,Comentarios: El lobo de wall street,Una serie de eventos desafortunados,Puntos: Una serie de eventos desafortunados,Comentarios: Una serie de eventos desafortunados
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
34,2020-07-14 11:21:10,2020-07-14 11:24:02,anonymous,NaN,NaN,NaN,Saturno,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,5.0,NaN,NaN,4,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2020-07-14 11:21:22,2020-07-14 11:24:17,anonymous,NaN,NaN,NaN,chaneke,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,4.0,NaN,NaN
36,2020-07-14 11:20:56,2020-07-14 11:24:36,anonymous,NaN,NaN,NaN,Lizette,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,2020-07-14 11:21:16,2020-07-14 11:24:36,anonymous,NaN,NaN,NaN,Alii,NaN,NaN,2.0,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
38,2020-07-14 11:21:18,2020-07-14 11:24:49,anonymous,NaN,NaN,NaN,Adolf,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,5,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,3,NaN,NaN,3.0,NaN,NaN,4,NaN,NaN,4.0,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


Seleccionar las columnas que contienen los datos necesarios para limpiar los datos que serán usados.

In [29]:
def seleccionar_columnas(x):
  csel = np.arange(9,246,3)
  usuarios1 = list(x.iloc[:,6])
  cnames1 = list(x.columns.values[csel])
  x = x[cnames1]
  x.index = usuarios1
  x[np.isnan(x)]=0
  return x

datan =  seleccionar_columnas(data)
datan.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


,Inteligencia Artificial,Super Man,E.T.,Interestelar,Mr. Nobody,Star Wars,V de Vendeta,Avatar,Contacto,El Origen (Inception),Matrix,I Origins,Lucy,La terminal,Diario de una pasión (The notebook),"Ghost, la sombra del amor",3 metros sobre el cielo,The Vow,Titanic,A star is born,A walk to remember,La la land,About time,Orgullo y prejuicio,Endless love,Fire up,Loco y estupido amor,La noche de los muertos vivientes,Exorcista,4to contacto,El conjuro,The ritual,La noche del demonio,El silencio de los inocentes,La monja,Saw,REC,Eso (It),La Bruja de Blair,Texas chainsaw,El rito,El exorcismo de Emily Rose,Your name,Coco,Intensa-mente,El viaje de Chihiro,Big hero 6 (Grandes heroes),Shrek,Toy story,Lilo y Stitch,Spider-Man,Fantastic Mr. Fox,South park,Trolls,Como entrenar a tu dragón,Locos de ira,Poderosa Afrodita,Como si fuera la primera vez,Mr. Bean,The proposal,"Crazy, Stupid, Love",Una esposa de mentiras,Scary movie,Deadpool,Superbad,Monthy Python,American Pie,Nosotros los nobles,La mascara,El señor de los anillos,Harry Potter,Blanca Nieves,Big fish,Narnia,El laberinto del fauno,The shape of water,Aquaman,El lobo de wall street,Una serie de eventos desafortunados
Saturno,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,3.0,0.0,4.0,1.0,1.0,0.0,3.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,4,5,0.0,5.0,5,5,3,4,4.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,4,5.0,4,4.0,5.0,2.0,0.0,0.0
chaneke,0.0,3.0,3.0,3.0,0.0,3.0,0.0,5,3.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,3.0,0.0,5.0,0.0,5.0,5.0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5,4,0.0,5.0,5,5,5,4,0.0,3.0,3.0,3.0,0.0,0.0,4.0,1.0,5.0,4.0,4.0,3.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,5.0,5,0.0,5,0.0,0.0,4.0,2.0,4.0
Lizette,0.0,1.0,3.0,0.0,0.0,3.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,4,4,0.0,5.0,4,4,5,4,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,4.0,3,0.0,4,0.0,0.0,3.0,0.0,0.0
Alii,2.0,5.0,1.0,3.0,0.0,4.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,5,0.0,0.0,5,4,5,4,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,3.0,5.0,3,0.0,3,0.0,0.0,3.0,0.0,1.0
Adolf,4.0,4.0,4.0,5.0,0.0,4.0,3.0,5,4.0,4.0,5.0,1.0,4.0,5.0,4.0,0.0,2.0,0.0,4.0,0.0,0.0,4.0,0.0,3.0,4.0,0.0,4.0,4.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,4.0,3.0,4.0,3.0,0.0,2.0,4.0,0.0,3,2,4.0,4.0,5,5,5,5,0.0,4.0,1.0,4.0,4.0,0.0,4.0,4.0,0.0,3.0,4.0,4.0,5.0,3.0,0.0,4.0,4.0,5.0,4.0,5.0,3,3.0,4,4.0,3.0,4.0,5.0,0.0


## **Datos Binarios**

Cambiar la calificación de estrellas (multiestado ordinal) a un rango de **me gusta** y **no me gusta**. Esta operación es necesaria porque los índices de similitud a probar son para datos binarios.

In [30]:
cnames = list(datan.columns.values)
fnames = np.array(datan.index)
for col in cnames:
    datan[col]=np.where(datan[col]>3,1,0)
datan.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Inteligencia Artificial,Super Man,E.T.,Interestelar,Mr. Nobody,Star Wars,V de Vendeta,Avatar,Contacto,El Origen (Inception),Matrix,I Origins,Lucy,La terminal,Diario de una pasión (The notebook),"Ghost, la sombra del amor",3 metros sobre el cielo,The Vow,Titanic,A star is born,A walk to remember,La la land,About time,Orgullo y prejuicio,Endless love,Fire up,Loco y estupido amor,La noche de los muertos vivientes,Exorcista,4to contacto,El conjuro,The ritual,La noche del demonio,El silencio de los inocentes,La monja,Saw,REC,Eso (It),La Bruja de Blair,Texas chainsaw,El rito,El exorcismo de Emily Rose,Your name,Coco,Intensa-mente,El viaje de Chihiro,Big hero 6 (Grandes heroes),Shrek,Toy story,Lilo y Stitch,Spider-Man,Fantastic Mr. Fox,South park,Trolls,Como entrenar a tu dragón,Locos de ira,Poderosa Afrodita,Como si fuera la primera vez,Mr. Bean,The proposal,"Crazy, Stupid, Love",Una esposa de mentiras,Scary movie,Deadpool,Superbad,Monthy Python,American Pie,Nosotros los nobles,La mascara,El señor de los anillos,Harry Potter,Blanca Nieves,Big fish,Narnia,El laberinto del fauno,The shape of water,Aquaman,El lobo de wall street,Una serie de eventos desafortunados
Saturno,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0
chaneke,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,1
Lizette,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
Alii,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Adolf,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,1,1,1,1,0,1,0,1,1,0,1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,0,1,1,0,1,1,0


Obtener la matriz de confusión (matriz de contingencia) y los indices de similitud.

In [31]:
cf_m = skm.confusion_matrix(datan.iloc[0,:],datan.iloc[1,:])

sim_simple = skm.accuracy_score(datan.iloc[0,:],datan.iloc[1,:])
#sim_simple_new = (cf_m[0,0]+cf_m[1,1])/np.sum(cf_m)
print('Simple : %0.4f'%sim_simple)

sim_jac = skm.jaccard_score(datan.iloc[0,:],datan.iloc[1,:])
#sim_jac = (cf_m[1,1])/(np.sum(cf_m)-cf_m[0,0])
print('Jaccard: %0.4f'%sim_jac)

Simple : 0.6709
Jaccard: 0.2778


Dependiendo de la librería usado, algunas pueden calcular las distancias en lugar de los índices de similitud directamente.

In [32]:
#%% Calculo de las distancias
d1 = sc.matching(datan.iloc[0,:],datan.iloc[1,:])
print('Simple : %0.4f'%d1)
d2 = sc.jaccard(datan.iloc[0,:],datan.iloc[1,:])
print('Jaccard: %0.4f'%d2)

Simple : 0.3291
Jaccard: 0.7222


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `matching` is deprecated!
spatial.distance.matching is deprecated in scipy 1.0.0; use spatial.distance.hamming instead.
  


Calcular los índices de similitud puede ser complicado debido a que la cantidad de operaciones está en función de la cantidad de datos que se tienen en el conjunto de datos.

In [33]:
#%% Calcular todas las combinaciones posibles
D1 = sc.pdist(datan,'matching')
D1 = sc.squareform(D1)

D2 = sc.pdist(datan,'jaccard')
D2 = sc.squareform(D2)

In [34]:
# Visualizar los índices calculados
pd.DataFrame(D1)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000000,0.329114,0.189873,0.253165,0.556962,0.291139,0.341772,0.417722,0.341772,0.303797,0.291139,0.379747,0.291139
1,0.329114,0.000000,0.240506,0.253165,0.430380,0.341772,0.367089,0.215190,0.291139,0.354430,0.240506,0.227848,0.392405
2,0.189873,0.240506,0.000000,0.113924,0.493671,0.227848,0.278481,0.329114,0.177215,0.240506,0.202532,0.291139,0.278481
3,0.253165,0.253165,0.113924,0.000000,0.430380,0.164557,0.291139,0.291139,0.164557,0.253165,0.240506,0.202532,0.341772
4,0.556962,0.430380,0.493671,0.430380,0.000000,0.367089,0.341772,0.392405,0.417722,0.455696,0.443038,0.379747,0.417722
5,0.291139,0.341772,0.227848,0.164557,0.367089,0.000000,0.177215,0.354430,0.253165,0.341772,0.278481,0.291139,0.329114
6,0.341772,0.367089,0.278481,0.291139,0.341772,0.177215,0.000000,0.379747,0.303797,0.341772,0.253165,0.316456,0.329114
7,0.417722,0.215190,0.329114,0.291139,0.392405,0.354430,0.379747,0.000000,0.354430,0.367089,0.379747,0.316456,0.455696
8,0.341772,0.291139,0.177215,0.164557,0.417722,0.253165,0.303797,0.354430,0.000000,0.265823,0.329114,0.341772,0.253165
9,0.303797,0.354430,0.240506,0.253165,0.455696,0.341772,0.341772,0.367089,0.265823,0.000000,0.316456,0.354430,0.291139


Tener todos los índices de similitud en una tabla puede confundir, y el uso dependerá de la finalidad de estas métricas. Por el momento nos enfocaremos a seleccionar un usuario en específico.

In [35]:
#%% Seleccionar un usuario y determinar los demás usuarios más parecidos
user = 1
D_user = D1[user]
D_user_sort = np.sort(D_user)
indx_user = np.argsort(D_user)

Recomendación de películas versión 1.

In [52]:
User = datan.loc[fnames[user]]
User_sim = datan.loc[fnames[indx_user[1]]]

indx_recomen = (User_sim ==1)&(User==0)
recomend1 = list(User.index[indx_recomen])
recomend1

['Interestelar',
 'Star Wars',
 'El Origen (Inception)',
 'Lucy',
 'The Vow',
 'A star is born',
 'La noche de los muertos vivientes',
 'Your name',
 'Trolls',
 'Como entrenar a tu dragón',
 'Scary movie']

Recomendación de películas versión 2.

In [53]:
User = datan.loc[fnames[user]]
User_sim = np.mean(datan.loc[fnames[indx_user[1:6]]],axis=0)
User_sim[User_sim<=0.5] = 0
User_sim[User_sim>0.5] = 1

indx_recomen = (User_sim ==1)&(User==0)
recomend2 = list(User.index[indx_recomen])
recomend2

['Star Wars', 'Como entrenar a tu dragón']

## **Datos Multiestado**

Seleccionar los datos sin necesidad de convertir la escala de estrellas a datos binarios.

In [54]:
datan =  seleccionar_columnas(data)
datan.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


,Inteligencia Artificial,Super Man,E.T.,Interestelar,Mr. Nobody,Star Wars,V de Vendeta,Avatar,Contacto,El Origen (Inception),Matrix,I Origins,Lucy,La terminal,Diario de una pasión (The notebook),"Ghost, la sombra del amor",3 metros sobre el cielo,The Vow,Titanic,A star is born,A walk to remember,La la land,About time,Orgullo y prejuicio,Endless love,Fire up,Loco y estupido amor,La noche de los muertos vivientes,Exorcista,4to contacto,El conjuro,The ritual,La noche del demonio,El silencio de los inocentes,La monja,Saw,REC,Eso (It),La Bruja de Blair,Texas chainsaw,El rito,El exorcismo de Emily Rose,Your name,Coco,Intensa-mente,El viaje de Chihiro,Big hero 6 (Grandes heroes),Shrek,Toy story,Lilo y Stitch,Spider-Man,Fantastic Mr. Fox,South park,Trolls,Como entrenar a tu dragón,Locos de ira,Poderosa Afrodita,Como si fuera la primera vez,Mr. Bean,The proposal,"Crazy, Stupid, Love",Una esposa de mentiras,Scary movie,Deadpool,Superbad,Monthy Python,American Pie,Nosotros los nobles,La mascara,El señor de los anillos,Harry Potter,Blanca Nieves,Big fish,Narnia,El laberinto del fauno,The shape of water,Aquaman,El lobo de wall street,Una serie de eventos desafortunados
Saturno,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,3.0,0.0,4.0,1.0,1.0,0.0,3.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,4,5,0.0,5.0,5,5,3,4,4.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,4,5.0,4,4.0,5.0,2.0,0.0,0.0
chaneke,0.0,3.0,3.0,3.0,0.0,3.0,0.0,5,3.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,3.0,0.0,5.0,0.0,5.0,5.0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5,4,0.0,5.0,5,5,5,4,0.0,3.0,3.0,3.0,0.0,0.0,4.0,1.0,5.0,4.0,4.0,3.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,5.0,5,0.0,5,0.0,0.0,4.0,2.0,4.0
Lizette,0.0,1.0,3.0,0.0,0.0,3.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,4,4,0.0,5.0,4,4,5,4,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,4.0,3,0.0,4,0.0,0.0,3.0,0.0,0.0
Alii,2.0,5.0,1.0,3.0,0.0,4.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,5,0.0,0.0,5,4,5,4,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,3.0,5.0,3,0.0,3,0.0,0.0,3.0,0.0,1.0
Adolf,4.0,4.0,4.0,5.0,0.0,4.0,3.0,5,4.0,4.0,5.0,1.0,4.0,5.0,4.0,0.0,2.0,0.0,4.0,0.0,0.0,4.0,0.0,3.0,4.0,0.0,4.0,4.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,4.0,3.0,4.0,3.0,0.0,2.0,4.0,0.0,3,2,4.0,4.0,5,5,5,5,0.0,4.0,1.0,4.0,4.0,0.0,4.0,4.0,0.0,3.0,4.0,4.0,5.0,3.0,0.0,4.0,4.0,5.0,4.0,5.0,3,3.0,4,4.0,3.0,4.0,5.0,0.0


Los índices de similitud más comunes están adaptados para funcionar con datos multiestado.

In [55]:
cf_m = skm.confusion_matrix(datan.iloc[0,:],datan.iloc[1,:])
sim_simple = skm.accuracy_score(datan.iloc[0,:],datan.iloc[1,:])
sim_jac = skm.jaccard_score(datan.iloc[0,:],datan.iloc[1,:],average='weighted')

Para los índices de similitud que no están adaptados, pueden convertirse los datos multiestado a datos binarios usando variables auxiliares.

In [64]:
#%% Generar variables dummies 
dum1 = pd.get_dummies(datan[cnames[0]])
#dum1 = pd.get_dummies(datan[cnames[0]],prefix=cnames[0])
dum1

,0.0,1.0,2.0,3.0,4.0,5.0
Saturno,1,0,0,0,0,0
chaneke,1,0,0,0,0,0
Lizette,1,0,0,0,0,0
Alii,0,0,1,0,0,0
Adolf,0,0,0,0,1,0
bob,1,0,0,0,0,0
Danny,1,0,0,0,0,0
Mar,1,0,0,0,0,0
Adriana,0,0,0,0,0,1
Alez,0,0,0,1,0,0


Obteniendo todas las variables auxiliares de todas las películas, hace que el conjunto de datos crezca considerablemente.

In [65]:
datan_dum = pd.get_dummies(datan[cnames[0]],prefix=cnames[0])
for col in cnames[1:]:
    tmp = pd.get_dummies(datan[col],prefix=col)
    datan_dum = datan_dum.join(tmp)
del tmp
datan_dum.head()

,Inteligencia Artificial_0.0,Inteligencia Artificial_1.0,Inteligencia Artificial_2.0,Inteligencia Artificial_3.0,Inteligencia Artificial_4.0,Inteligencia Artificial_5.0,Super Man_0.0,Super Man_1.0,Super Man_2.0,Super Man_3.0,Super Man_4.0,Super Man_5.0,E.T._0.0,E.T._1.0,E.T._3.0,E.T._4.0,E.T._5.0,Interestelar_0.0,Interestelar_3.0,Interestelar_4.0,Interestelar_5.0,Mr. Nobody_0.0,Mr. Nobody_1.0,Mr. Nobody_2.0,Mr. Nobody_3.0,Star Wars_0.0,Star Wars_2.0,Star Wars_3.0,Star Wars_4.0,Star Wars_5.0,V de Vendeta_0.0,V de Vendeta_1.0,V de Vendeta_3.0,V de Vendeta_5.0,Avatar_2,Avatar_3,Avatar_4,Avatar_5,Contacto_0.0,Contacto_1.0,...,El señor de los anillos_4.0,Harry Potter_0.0,Harry Potter_4.0,Harry Potter_5.0,Blanca Nieves_2,Blanca Nieves_3,Blanca Nieves_4,Blanca Nieves_5,Big fish_0.0,Big fish_1.0,Big fish_3.0,Big fish_4.0,Big fish_5.0,Narnia_2,Narnia_3,Narnia_4,Narnia_5,El laberinto del fauno_0.0,El laberinto del fauno_2.0,El laberinto del fauno_3.0,El laberinto del fauno_4.0,El laberinto del fauno_5.0,The shape of water_0.0,The shape of water_3.0,The shape of water_4.0,The shape of water_5.0,Aquaman_0.0,Aquaman_2.0,Aquaman_3.0,Aquaman_4.0,Aquaman_5.0,El lobo de wall street_0.0,El lobo de wall street_2.0,El lobo de wall street_3.0,El lobo de wall street_4.0,El lobo de wall street_5.0,Una serie de eventos desafortunados_0.0,Una serie de eventos desafortunados_1.0,Una serie de eventos desafortunados_4.0,Una serie de eventos desafortunados_5.0
Saturno,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0
chaneke,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
Lizette,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0
Alii,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
Adolf,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0


## **Datos Cuantitativos**

Los datos cuantitativos pueden procesarse directamente por medio de otros índices de similitud diseñados para este tipo de datos.

In [66]:
datan =  seleccionar_columnas(data)
datan.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


,Inteligencia Artificial,Super Man,E.T.,Interestelar,Mr. Nobody,Star Wars,V de Vendeta,Avatar,Contacto,El Origen (Inception),Matrix,I Origins,Lucy,La terminal,Diario de una pasión (The notebook),"Ghost, la sombra del amor",3 metros sobre el cielo,The Vow,Titanic,A star is born,A walk to remember,La la land,About time,Orgullo y prejuicio,Endless love,Fire up,Loco y estupido amor,La noche de los muertos vivientes,Exorcista,4to contacto,El conjuro,The ritual,La noche del demonio,El silencio de los inocentes,La monja,Saw,REC,Eso (It),La Bruja de Blair,Texas chainsaw,El rito,El exorcismo de Emily Rose,Your name,Coco,Intensa-mente,El viaje de Chihiro,Big hero 6 (Grandes heroes),Shrek,Toy story,Lilo y Stitch,Spider-Man,Fantastic Mr. Fox,South park,Trolls,Como entrenar a tu dragón,Locos de ira,Poderosa Afrodita,Como si fuera la primera vez,Mr. Bean,The proposal,"Crazy, Stupid, Love",Una esposa de mentiras,Scary movie,Deadpool,Superbad,Monthy Python,American Pie,Nosotros los nobles,La mascara,El señor de los anillos,Harry Potter,Blanca Nieves,Big fish,Narnia,El laberinto del fauno,The shape of water,Aquaman,El lobo de wall street,Una serie de eventos desafortunados
Saturno,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,3.0,0.0,4.0,1.0,1.0,0.0,3.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,4,5,0.0,5.0,5,5,3,4,4.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,4,5.0,4,4.0,5.0,2.0,0.0,0.0
chaneke,0.0,3.0,3.0,3.0,0.0,3.0,0.0,5,3.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,3.0,0.0,5.0,0.0,5.0,5.0,0.0,5.0,5.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5,4,0.0,5.0,5,5,5,4,0.0,3.0,3.0,3.0,0.0,0.0,4.0,1.0,5.0,4.0,4.0,3.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,5.0,5,0.0,5,0.0,0.0,4.0,2.0,4.0
Lizette,0.0,1.0,3.0,0.0,0.0,3.0,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,4,4,0.0,5.0,4,4,5,4,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,4.0,3,0.0,4,0.0,0.0,3.0,0.0,0.0
Alii,2.0,5.0,1.0,3.0,0.0,4.0,0.0,4,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,5,0.0,0.0,5,4,5,4,0.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,3.0,5.0,3,0.0,3,0.0,0.0,3.0,0.0,1.0
Adolf,4.0,4.0,4.0,5.0,0.0,4.0,3.0,5,4.0,4.0,5.0,1.0,4.0,5.0,4.0,0.0,2.0,0.0,4.0,0.0,0.0,4.0,0.0,3.0,4.0,0.0,4.0,4.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,4.0,3.0,4.0,3.0,0.0,2.0,4.0,0.0,3,2,4.0,4.0,5,5,5,5,0.0,4.0,1.0,4.0,4.0,0.0,4.0,4.0,0.0,3.0,4.0,4.0,5.0,3.0,0.0,4.0,4.0,5.0,4.0,5.0,3,3.0,4,4.0,3.0,4.0,5.0,0.0


La distancia Euclideana puede considerarse como una métrica de similitud porque mide la distancia geométrica entre dos puntos en un espacio multidimensional.

In [68]:
#%% Distancia Euclideana
D1 = sc.pdist(datan,'euclidean')
D1 = sc.squareform(D1)
pd.DataFrame(D1)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000000,22.561028,17.117243,19.287302,25.922963,20.248457,20.371549,24.041631,21.071308,19.874607,21.095023,24.494897,19.000000
1,22.561028,0.000000,18.601075,17.972201,22.068076,19.313208,20.880613,17.521415,18.788294,21.260292,21.166010,17.972201,22.135944
2,17.117243,18.601075,0.000000,12.688578,25.632011,17.117243,21.400935,21.283797,15.000000,20.149442,19.646883,21.236761,19.339080
3,19.287302,17.972201,12.688578,0.000000,23.021729,14.696938,19.209373,20.445048,13.784049,19.519221,18.520259,17.320508,20.322401
4,25.922963,22.068076,25.632011,23.021729,0.000000,20.149442,16.217275,23.748684,23.108440,22.649503,22.912878,22.538855,18.357560
5,20.248457,19.313208,17.117243,14.696938,20.149442,0.000000,14.730920,22.271057,17.720045,22.113344,19.261360,18.654758,18.138357
6,20.371549,20.880613,21.400935,19.209373,16.217275,14.730920,0.000000,23.173260,21.047565,21.260292,18.384776,20.074860,17.320508
7,24.041631,17.521415,21.283797,20.445048,23.748684,22.271057,23.173260,0.000000,22.271057,22.472205,25.980762,23.579652,25.159491
8,21.071308,18.788294,15.000000,13.784049,23.108440,17.720045,21.047565,22.271057,0.000000,17.406895,23.130067,23.021729,18.627936
9,19.874607,21.260292,20.149442,19.519221,22.649503,22.113344,21.260292,22.472205,17.406895,0.000000,23.151674,23.086793,18.867962


La distancia coseno mide el ángulo entre dos punto en un espacio multidimensional.

In [69]:
#%% Distancia coseno
D2 = sc.pdist(datan,'cosine')
D2 = sc.squareform(D2)
pd.DataFrame(D2)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000000,0.419278,0.356608,0.435947,0.441458,0.440674,0.346192,0.418481,0.486297,0.373157,0.411980,0.527312,0.339145
1,0.419278,0.000000,0.284248,0.263291,0.287467,0.305070,0.313147,0.196401,0.289033,0.349658,0.342169,0.237702,0.378524
2,0.356608,0.284248,0.000000,0.234094,0.447432,0.371603,0.416095,0.320372,0.291089,0.432330,0.395658,0.432909,0.392046
3,0.435947,0.263291,0.234094,0.000000,0.336062,0.259330,0.318398,0.291875,0.234150,0.389945,0.338192,0.266454,0.422457
4,0.441458,0.287467,0.447432,0.336062,0.000000,0.241917,0.150471,0.315546,0.340249,0.317039,0.323296,0.306636,0.197630
5,0.440674,0.305070,0.371603,0.259330,0.241917,0.000000,0.173030,0.356123,0.353471,0.472470,0.348410,0.304704,0.314047
6,0.346192,0.313147,0.416095,0.318398,0.150471,0.173030,0.000000,0.349930,0.379427,0.356182,0.262154,0.301604,0.234543
7,0.418481,0.196401,0.320372,0.291875,0.315546,0.356123,0.349930,0.000000,0.357870,0.347111,0.464691,0.369693,0.438619
8,0.486297,0.289033,0.291089,0.234150,0.340249,0.353471,0.379427,0.357870,0.000000,0.293562,0.514086,0.478670,0.335897
9,0.373157,0.349658,0.432330,0.389945,0.317039,0.472470,0.356182,0.347111,0.293562,0.000000,0.458825,0.434750,0.309024


El índice de correlación tambien puede ser usada para medir la similitud entre muestras en un conjunto de datos.

In [70]:
#%% Indice correlación
D3 = sc.pdist(datan,'correlation')
D3 = sc.squareform(D3)
pd.DataFrame(D3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000000,0.743637,0.529907,0.683058,0.954067,0.724900,0.653448,0.731826,0.784241,0.637583,0.670505,0.861828,0.667473
1,0.743637,0.000000,0.433470,0.433168,0.709449,0.542851,0.682163,0.380700,0.498986,0.657348,0.600931,0.416769,0.894533
2,0.529907,0.433470,0.000000,0.337701,0.786301,0.554572,0.680248,0.490864,0.427646,0.663654,0.587103,0.644983,0.648385
3,0.683058,0.433168,0.337701,0.000000,0.605749,0.407196,0.550257,0.478521,0.362086,0.633277,0.527343,0.416450,0.786163
4,0.954067,0.709449,0.786301,0.605749,0.000000,0.450798,0.433239,0.759601,0.667878,0.718064,0.641557,0.606458,0.688350
5,0.724900,0.542851,0.554572,0.407196,0.450798,0.000000,0.309115,0.626542,0.573088,0.813467,0.570277,0.500894,0.619276
6,0.653448,0.682163,0.680248,0.550257,0.433239,0.309115,0.000000,0.744929,0.703108,0.728687,0.479089,0.562846,0.642378
7,0.731826,0.380700,0.490864,0.478521,0.759601,0.626542,0.744929,0.000000,0.613449,0.641961,0.808534,0.645195,1.015164
8,0.784241,0.498986,0.427646,0.362086,0.667878,0.573088,0.703108,0.613449,0.000000,0.492242,0.825249,0.771475,0.639563
9,0.637583,0.657348,0.663654,0.633277,0.718064,0.813467,0.728687,0.641961,0.492242,0.000000,0.780227,0.742737,0.664600
